# Hyperparameter Tuning using HyperDrive

In [3]:
import logging
import os
import csv
import joblib

import azureml.core
from azureml.core.compute_target import ComputeTargetException
from azureml.automl.core.featurization import FeaturizationConfig
from azureml.core.dataset import Dataset
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.interpret import ExplanationClient
from azureml.automl.runtime.onnx_convert import OnnxConverter
from azureml.data.dataset_factory import TabularDatasetFactory
from azureml.core import Environment, ScriptRunConfig
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.model import Model
from azureml.core import Environment, ScriptRunConfig
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
from azureml.widgets import RunDetails
from azureml.core.run import Run
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
import pkg_resources

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.19.0


## Initialize Workspace and Create an Azure ML experiment

In [5]:
ws = Workspace.from_config()
experiment_name = 'heart-failure-hyperdrive-experiment'

experiment=Experiment(ws, experiment_name)

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = experiment.start_logging()

Workspace name: quick-starts-ws-135902
Azure region: southcentralus
Subscription id: cdbe0b43-92a0-4715-838a-f2648cc7ad21
Resource group: aml-quickstarts-135902


## Dataset

In [7]:
# Try to load the dataset from the Workspace. Otherwise, create it from the file
dataset_key = "Heart-Failure-Prediction-Dataset"
description_text = "Heart Failure Prediction Dataset"

if dataset_key in ws.datasets.keys(): 
    dataset = ws.datasets[dataset_key]
    print("Found dataset")
else:
    # Create AML Dataset and register it into Workspace
    dataset_url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/00519/heart_failure_clinical_records_dataset.csv'
    dataset = Dataset.Tabular.from_delimited_files(dataset_url)        
    #Register Dataset in Workspace
    dataset = dataset.register(workspace=ws,
                                name=dataset_key,
                                description=description_text)


df = dataset.to_pandas_dataframe()
df.describe()

Found dataset


,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
count,299.00,299.00,299.00,299.00,299.00,299.00,299.00,299.00,299.00,299.00,299.00,299.00,299.00
mean,60.83,0.43,581.84,0.42,38.08,0.35,263358.03,1.39,136.63,0.65,0.32,130.26,0.32
std,11.89,0.50,970.29,0.49,11.83,0.48,97804.24,1.03,4.41,0.48,0.47,77.61,0.47
min,40.00,0.00,23.00,0.00,14.00,0.00,25100.00,0.50,113.00,0.00,0.00,4.00,0.00
25%,51.00,0.00,116.50,0.00,30.00,0.00,212500.00,0.90,134.00,0.00,0.00,73.00,0.00
50%,60.00,0.00,250.00,0.00,38.00,0.00,262000.00,1.10,137.00,1.00,0.00,115.00,0.00
75%,70.00,1.00,582.00,1.00,45.00,1.00,303500.00,1.40,140.00,1.00,1.00,203.00,1.00
max,95.00,1.00,7861.00,1.00,80.00,1.00,850000.00,9.40,148.00,1.00,1.00,285.00,1.00


## Create or Attach an AmlCompute cluster

In [8]:
from azureml.core.compute import ComputeTarget, AmlCompute

cluster_name = "cpu-cluster"

# Check if the compute target exists
try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing cluster, use it.')
except:
    # If not, create it
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS12_V2', max_nodes=5)
    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)
compute_target.wait_for_completion(show_output=True)

print(compute_target.get_status().serialize())

Found existing cluster, use it.

Running
{'errors': [], 'creationTime': '2021-01-24T14:59:14.660177+00:00', 'createdBy': {'userObjectId': 'de62da4e-e66d-430d-a4a9-1c7ffdd05d75', 'userTenantId': '660b3398-b80e-49d2-bc5b-ac1dc93b5254', 'userName': 'ODL_User 135902'}, 'modifiedTime': '2021-01-24T15:02:16.464259+00:00', 'state': 'Running', 'vmSize': 'STANDARD_DS12_V2'}


## Create conda dependencies file 

In [9]:
%%writefile conda_dependencies.yml

dependencies:
- python=3.6.2
- scikit-learn
- numpy
- pip:
  - azureml-defaults

Writing conda_dependencies.yml


## Create sklearn environment

In [10]:
from azureml.core import Environment
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')

## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [12]:
# Create an early termination policy. This is not required if you are using Bayesian sampling.
early_termination_policy = BanditPolicy(slack_factor = 0.1, evaluation_interval = 1, delay_evaluation=5)

#Create the different params that you will be using during training
ps = RandomParameterSampling(
    {
        "--C" :        choice(0.001,0.01,0.1, 0.5, 1,1.5,10,20,50,100,200,500,1000),
        "--max_iter" : choice(25,50,75,100,200,300)
    }
)

#Create estimator and hyperdrive config
src = ScriptRunConfig(source_directory='.',
                      script='train.py',
                      compute_target=compute_target,
                      environment=sklearn_env)

hyperdrive_run_config =  HyperDriveConfig(
    hyperparameter_sampling = ps, 
    policy = early_termination_policy,
    primary_metric_name = 'Accuracy',
    primary_metric_goal = PrimaryMetricGoal.MAXIMIZE, 
    max_total_runs = 100,
    max_concurrent_runs = 4,
    run_config = src
)

In [15]:
#Submit experiment
hyperdrive_run = experiment.submit(hyperdrive_run_config, show_output=True)

## Run Details

In [16]:
RunDetails(hyperdrive_run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [17]:
hyperdrive_run.get_status()

'Running'

In [18]:
hyperdrive_run.wait_for_completion(show_output=True)

RunId: HD_9a5cea5a-9b16-4133-86af-da3dc6a70da3
Web View: https://ml.azure.com/experiments/heart-failure-hyperdrive-experiment/runs/HD_9a5cea5a-9b16-4133-86af-da3dc6a70da3?wsid=/subscriptions/cdbe0b43-92a0-4715-838a-f2648cc7ad21/resourcegroups/aml-quickstarts-135902/workspaces/quick-starts-ws-135902

Streaming azureml-logs/hyperdrive.txt

"<START>[2021-01-24T15:22:21.560135][API][INFO]Experiment created<END>\n"<START>[2021-01-24T15:22:23.0040585Z][SCHEDULER][INFO]The execution environment is being prepared. Please be patient as it can take a few minutes.<END>"<START>[2021-01-24T15:22:24.801422][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space<END>\n""<START>[2021-01-24T15:22:25.172135][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution target.<END>\n"

Execution Summary
RunId: HD_9a5cea5a-9b16-4133-86af-da3dc6a70da3
Web View: https://ml.azure.com/experiments/heart-failure-hyperdrive-experiment/runs/HD_9a5cea5a-9b16-4133-86a

{'runId': 'HD_9a5cea5a-9b16-4133-86af-da3dc6a70da3',
 'target': 'cpu-cluster',
 'status': 'Completed',
 'startTimeUtc': '2021-01-24T15:22:21.329816Z',
 'endTimeUtc': '2021-01-24T16:07:19.482072Z',
 'error': {'error': {'code': 'UserError',
   'message': 'User errors were found in at least one of the child runs.',
   'messageParameters': {},
   'details': []},
  'time': '0001-01-01T00:00:00.000Z'},
 'properties': {'primary_metric_config': '{"name": "Accuracy", "goal": "maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': '3898863e-9e2c-4d94-b92c-8643bc86cc1a',
  'score': '0.8333333333333334',
  'best_child_run_id': 'HD_9a5cea5a-9b16-4133-86af-da3dc6a70da3_5',
  'best_metric_status': 'Succeeded'},
 'inputDatasets': [],
 'outputDatasets': [],
 'logFiles': {'azureml-logs/hyperdrive.txt': 'https://mlstrg135902.blob.core.windows.net/azureml/ExperimentRun/dcid.HD_9a5cea5a-9b16-4133-86af-da3dc6a70da

## Best Model

In [19]:
best_run = hyperdrive_run.get_best_run_by_primary_metric()
print('Best Run Id:', best_run.id, sep='\n')
print('Best Run Metrics:', best_run.get_metrics(), sep='\n')
print('Best Run Properties:', best_run.get_properties(), sep='\n')
print('Best Run Parameters:', best_run.get_details()['runDefinition']['arguments'], sep='\n')
print('Best Run File names:', best_run.get_file_names(), sep='\n')

Best Run Id:
HD_9a5cea5a-9b16-4133-86af-da3dc6a70da3_5
Best Run Metrics:
{'Regularization Strength:': 0.5, 'Max iterations:': 50, 'Accuracy': 0.8333333333333334}
Best Run Properties:
{'_azureml.ComputeTargetType': 'amlcompute', 'ContentSnapshotId': '3898863e-9e2c-4d94-b92c-8643bc86cc1a', 'ProcessInfoFile': 'azureml-logs/process_info.json', 'ProcessStatusFile': 'azureml-logs/process_status.json'}
Best Run Parameters:
['--C', '0.5', '--max_iter', '50']
Best Run File names:
['azureml-logs/55_azureml-execution-tvmps_1880dcee40be74a7ba5dfb150ae271442a7d865e5525b76124a50dc1d5eccab9_d.txt', 'azureml-logs/65_job_prep-tvmps_1880dcee40be74a7ba5dfb150ae271442a7d865e5525b76124a50dc1d5eccab9_d.txt', 'azureml-logs/70_driver_log.txt', 'azureml-logs/75_job_post-tvmps_1880dcee40be74a7ba5dfb150ae271442a7d865e5525b76124a50dc1d5eccab9_d.txt', 'azureml-logs/process_info.json', 'azureml-logs/process_status.json', 'logs/azureml/93_azureml.log', 'logs/azureml/dataprep/backgroundProcess.log', 'logs/azureml/dat

In [20]:
#Save the best model
hyperdrive_model = best_run.register_model(model_name = 'hyperdrive_model', model_path = './outputs/model.joblib')
print(best_run)

Run(Experiment: heart-failure-hyperdrive-experiment,
Id: HD_9a5cea5a-9b16-4133-86af-da3dc6a70da3_5,
Type: azureml.scriptrun,
Status: Completed)


In [21]:
best_run.download_file("/outputs/model.joblib", "./outputs/model.joblib")

## Cleanup

In [ ]:
compute_target.delete()